<a href="https://colab.research.google.com/github/Alex-Ukraine/tf2-fcc/blob/master/tenth_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [165]:
import tensorflow as tf

import numpy as np
import os
import time

In [166]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [167]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving dan.txt to dan.txt


In [168]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 461979 characters


In [169]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿How many friends does one person need? 

Acknowledgements

This volume had its origins in a series of popular science articles that I wrote for New Scientist magazine (mostly between 1994 and 2006) and the Scotsman newspaper (between 2005 and 20


In [170]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

97 unique characters


In [171]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [172]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '$' :   4,
  '(' :   5,
  ')' :   6,
  '*' :   7,
  '+' :   8,
  ',' :   9,
  '-' :  10,
  '.' :  11,
  '/' :  12,
  '0' :  13,
  '1' :  14,
  '2' :  15,
  '3' :  16,
  '4' :  17,
  '5' :  18,
  '6' :  19,
  ...
}


In [173]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\ufeffHow many fri' ---- characters mapped to int ---- > [96 34 69 77  2 67 55 68 79  2 60 72 63]


In [174]:
# The maximum length sentence we want for a single input in characters
seq_length = 50
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

﻿
H
o
w
 


In [175]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\ufeffHow many friends does one person need? \r\n\r\nAcknowl'
'edgements\r\n\r\nThis volume had its origins in a serie'
's of popular science articles that I wrote for New '
'Scientist magazine (mostly between 1994 and 2006) a'
'nd the Scotsman newspaper (between 2005 and 2008). '


In [176]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [177]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeffHow many friends does one person need? \r\n\r\nAcknow'
Target data: 'How many friends does one person need? \r\n\r\nAcknowl'


In [178]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 96 ('\ufeff')
  expected output: 34 ('H')
Step    1
  input: 34 ('H')
  expected output: 69 ('o')
Step    2
  input: 69 ('o')
  expected output: 77 ('w')
Step    3
  input: 77 ('w')
  expected output: 2 (' ')
Step    4
  input: 2 (' ')
  expected output: 67 ('m')


In [179]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 50), (64, 50)), types: (tf.int64, tf.int64)>

In [180]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [181]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [182]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [183]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 50, 97) # (batch_size, sequence_length, vocab_size)


In [184]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (64, None, 256)           24832     
_________________________________________________________________
gru_9 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_9 (Dense)              (64, None, 97)            99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [185]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [186]:
sampled_indices

array([88, 30, 35, 92, 76, 56, 86, 92, 82, 51,  1, 17, 63, 59, 17,  7, 43,
       77, 13, 67, 42, 78, 88, 35, 66, 69, 62, 59, 12, 63, 91, 37, 58, 31,
       24, 46,  4, 61,  6, 80,  5,  7, 67, 84, 22,  2, 53, 25,  2, 80])

In [187]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'been. We witnessed a genuine scientific revolution'

Next Char Predictions: 
 'ïDIüvbêüàY\r4ie4*Qw0mPxïIlohe/iöKdE;T$g)z(*mè9 [= z'


In [188]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 50, 97)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5760612


In [189]:
model.compile(optimizer='adam', loss=loss)

In [190]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [191]:
EPOCHS=100

In [192]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
141/141 [==============================] - 4s 28ms/step - loss: 2.7494
Epoch 2/100
141/141 [==============================] - 4s 28ms/step - loss: 2.1241
Epoch 3/100
141/141 [==============================] - 4s 28ms/step - loss: 1.8689
Epoch 4/100
141/141 [==============================] - 4s 28ms/step - loss: 1.6676
Epoch 5/100
141/141 [==============================] - 4s 28ms/step - loss: 1.5346
Epoch 6/100
141/141 [==============================] - 4s 28ms/step - loss: 1.4423
Epoch 7/100
141/141 [==============================] - 4s 28ms/step - loss: 1.3689
Epoch 8/100
141/141 [==============================] - 4s 28ms/step - loss: 1.3109
Epoch 9/100
141/141 [==============================] - 4s 29ms/step - loss: 1.2576
Epoch 10/100
141/141 [==============================] - 4s 28ms/step - loss: 1.2088
Epoch 11/100
141/141 [==============================] - 4s 28ms/step - loss: 1.1593
Epoch 12/100
141/141 [==============================] - 4s 29ms/step - loss: 1.1109
E

In [193]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_100'

In [194]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [195]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (1, None, 256)            24832     
_________________________________________________________________
gru_10 (GRU)                 (1, None, 1024)           3938304   
_________________________________________________________________
dense_10 (Dense)             (1, None, 97)             99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [196]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [197]:
print(generate_text(model, start_string=u"was"))

was brain

so complete Darwin’. 

Hence, with the ideclimitive by good for you

What tec societion is transmits in a harvest back to fight. They alive mitionary because their possetion, to pussionally, God wants casefay?’ was only forced the free birded local backqup or out fasignuses can rescues of spearch. It cassior of the oldest don’t tell-unking, we know that of co-of which placts that is thquirden as marterable burial grooming. We bs the female life is alllue as orromance they troughly of treen to the same smale coble could really by exception of 150 because they scholar and so named Braskfix, they would expect them alwaysally structured in an onalog. Nearly been lifed are distrefect thats were didebabout the qualtations of the reason is viller (fictions that men than frequency of our community, the but rulkerfor martoric colonishes with sught stage by vision of comeology. Foot ever from the greates a big brains true: even a fast size of Licting breath of young chest the first
